# 08 Demo: Trajectory Analysis
UW Geospatial Data Analysis  
CEE498/CEWA599  
David Shean  

## Pandas Time Series
* https://pandas.pydata.org/docs/user_guide/timeseries.html
* https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html
* https://pandas.pydata.org/docs/user_guide/timeseries.html#overview

## Topics to cover
* Ingesting and DateTime Index
* Resampling
* Indexing

In [ ]:
import os
import pandas as pd
import geopandas as gpd

In [ ]:
import hvplot.pandas

In [ ]:
datadir = './data'
fn1=os.path.join(datadir, 'Shean_OASfM_20130909_PosnPnt.csv')
fn2=os.path.join(datadir, 'Shean_OASfM_20130910_PosnPnt.csv')

In [ ]:
df = pd.read_csv(fn1)

In [ ]:
df

In [ ]:
pd.to_datetime(df['GPS Date'] + ' ' + df['GPS Time'])

In [ ]:
df = pd.read_csv(fn1, parse_dates=[['GPS Date', 'GPS Time']])

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df.drop(columns=['Latitude.1', 'Longitude.1'], inplace=True)

In [ ]:
df.rename(columns={'GPS Date_GPS Time':'GPS DateTime'}, inplace=True)

In [ ]:
df.set_index('GPS DateTime', inplace=True)

In [ ]:
df.head()

In [ ]:
#Define function to do the above wrangling
def parse_trimblexh_export(fn):
    df = pd.read_csv(fn, parse_dates=[['GPS Date', 'GPS Time']])
    df.drop(columns=['Latitude.1', 'Longitude.1'], inplace=True)
    df.rename(columns={'GPS Date_GPS Time':'GPS DateTime'}, inplace=True)
    df.set_index('GPS DateTime', inplace=True)
    return df

In [ ]:
df1 = parse_trimblexh_export(fn1)
df2 = parse_trimblexh_export(fn2)

In [ ]:
print(df1.shape)
print(df2.shape)

In [ ]:
df = pd.concat([df1, df2])

In [ ]:
df

In [ ]:
df1.columns

In [ ]:
#%matplotlib widget

In [ ]:
df1['HAE'].plot();

In [ ]:
df1['HAE'].hvplot()

In [ ]:
df.hvplot()

In [ ]:
df['2013-09-10 20:26:57':'2013-09-10 20:26:59']

In [ ]:
gdf1 = gpd.GeoDataFrame(df1, crs='EPSG:4326', geometry=gpd.points_from_xy(df1['Longitude'], df1['Latitude']))

In [ ]:
gdf = gpd.GeoDataFrame(df, crs='EPSG:4326', geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']))

In [ ]:
gdf1

In [ ]:
gdf1.hvplot(aspect='equal')

In [ ]:
gdf1_utm = gdf1.to_crs('EPSG:32610')

In [ ]:
gdf1_utm.hvplot(aspect='equal', c='HAE')

In [ ]:
#https://stackoverflow.com/questions/40452759/pandas-latitude-longitude-to-distance-between-successive-rows

In [ ]:
gdf1_utm.head()

## Horizontal Velocity (ground speed)
* Could also look at 3-D vector displacement, vertical velocity, etc.

In [ ]:
gdf1_utm.distance(gdf1_utm.shift(1))

In [ ]:
gdf1_utm['dm'] = gdf1_utm.distance(gdf1_utm.shift(1))

In [ ]:
gdf1_utm['dm'].max()

In [ ]:
gdf1_utm.index

In [ ]:
gdf1_utm.index.to_series().diff()

In [ ]:
gdf1_utm['dt'] = gdf1_utm.index.to_series().diff().dt.total_seconds()

In [ ]:
gdf1_utm['dt']

In [ ]:
gdf1_utm['v'] = gdf1_utm['dm']/gdf1_utm['dt']

In [ ]:
gdf1_utm['v'].hvplot()

In [ ]:
gdf1_utm['a'] = gdf1_utm['v'].diff()

In [ ]:
from geoviews import tile_sources as gvts
#map_tiles = gvts.StamenTerrain
map_tiles = gvts.EsriImagery

In [ ]:
map_tiles

In [ ]:
#kw = {'width':600, 'height':600, 'data_aspect':1, 'alpha':1.0}
kw = {'width':400, 'height':400, 'data_aspect':1, 'alpha':1.0}
kw['colorbar'] = True
kw['cmap'] = 'inferno'
#kw['hover'] = False
#kw['datashade'] = True

### Overlay points on map tiles (subset of points)
* To combine layers in a single plot using `holoviews`, you use the asterisk (*) to overlay the two objects
    * You can use the plus sign (+) to build a layout with two separate subplots 
* Currently need to use the `geo=True` option and revert back to the `glas_gdf` GeoDataFrame with lat/lon geometry, not our reprojected points in `glas_gdf_aea`.  
    * In principle, hvplot/geoviews should work with our projected points, but there are some residual issues, and it appears that some underlying code somewhere in the stack is assuming lat/lon

In [ ]:
map_tiles * gdf1_utm[::3].to_crs('EPSG:3857').hvplot(c='HAE', size=1, **kw)

In [ ]:
gdf1_utm[::3].shape

## Using datashader to efficiently render all points

In [ ]:
map_tiles * gdf1_utm.to_crs('EPSG:3857').hvplot(c='HAE', datashade=True, dynamic=True, **kw)

## Other useful analysis
* Distance from starting point
* Time spent in different elevation bins
* Altitude above ground - sample DEM
* Geotagging photos based on timestamp